In [1]:
import pandas as pd
import numpy as np
import torch
from torch import cuda
from torch.utils.data import Dataset, DataLoader
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
from load_data import initialize_data
from reading_datasets import read_task5
from labels_to_ids import task5_labels_to_ids
import time
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
def train(epoch, train_loader, model, optimizer, device, grad_step = 1, max_grad_norm = 10):
    tr_loss, tr_accuracy = 0, 0
    tr_precision, tr_recall = 0, 0
    tr_f1score = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    optimizer.zero_grad()
    
    for idx, batch in enumerate(train_loader):
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        if (idx + 1) % 20 == 0:
            print('FINSIHED BATCH:', idx, 'of', len(train_loader))

        #loss, tr_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
        output = model(input_ids=ids, attention_mask=mask, labels=labels)
        tr_loss += output[0]

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = output[1].view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
        
        # Compute Precision
        tmp_tr_precision = precision_score(labels.cpu().numpy(), predictions.cpu().numpy(), labels=[0,1,2], average = None, zero_division=0 )[2]
        tr_precision += tmp_tr_precision
        
        # Compute Recall
        tmp_tr_recall = recall_score(labels.cpu().numpy(), predictions.cpu().numpy(), labels=[0,1,2], average = None, zero_division=0 )[2]
        tr_recall += tmp_tr_recall
        
        # Compute f1score
        tmp_tr_f1score = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(),labels=[0,1,2], average=None, zero_division=0)[2]
        tr_f1score += tmp_tr_f1score
    

        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=max_grad_norm
        )
        
        # backward pass
        output['loss'].backward()
        if (idx + 1) % grad_step == 0:
            optimizer.step()
            optimizer.zero_grad()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    tr_precision = tr_precision / nb_tr_steps
    tr_recall = tr_recall / nb_tr_steps
    tr_f1score= tr_f1score / nb_tr_steps
    #print(f"Training loss epoch: {epoch_loss}")
    #print(f"Training accuracy epoch: {tr_accuracy}")

    return model

In [3]:
def testing(model, testing_loader, labels_to_ids, device):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    eval_precision, eval_recall = 0, 0
    eval_f1score = 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
     
    
    ids_to_labels = dict((v,k) for k,v in labels_to_ids.items())

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            
            #loss, eval_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
            output = model(input_ids=ids, attention_mask=mask, labels=labels)

            eval_loss += output['loss'].item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = output[1].view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy
            
            # Compute Precision
            tmp_eval_precision = precision_score(labels.cpu().numpy(), predictions.cpu().numpy(), labels=[0,1,2], average = None, zero_division=0)[2]
            eval_precision += tmp_eval_precision
            
            # Compute Recall
            tmp_eval_recall = recall_score(labels.cpu().numpy(), predictions.cpu().numpy(), labels=[0,1,2], average = None, zero_division=0)[2]
            eval_recall += tmp_eval_recall
            
            # Compute f1score
            tmp_eval_f1score = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(),labels=[0,1,2], average=None, zero_division=0)[2]
            eval_f1score += tmp_eval_f1score

    labels = [ids_to_labels[id.item()] for id in eval_labels]
    predictions = [ids_to_labels[id.item()] for id in eval_preds]
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    eval_precision = eval_precision / nb_eval_steps
    eval_recall = eval_recall / nb_eval_steps
    eval_f1score = eval_f1score / nb_eval_steps
    #print(f"Validation Loss: {eval_loss}")
    #print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions, eval_accuracy, eval_precision, eval_recall, eval_f1score

In [4]:
def main(n_epochs, model_name, model_save_flag, model_save_location, model_load_flag, model_load_location):
    #Initialization training parameters
    max_len = 256
    batch_size = 32
    grad_step = 1
    learning_rate = 1e-05
    initialization_input = (max_len, batch_size)

    #Reading datasets and initializing data loaders
    dataset_location = '../2022.07.07_task5/'

    train_data = read_task5(dataset_location , split = 'train_over_under_sample')
    dev_data = read_task5(dataset_location , split = 'dev')
    #test_data = read_task5(dataset_location , split = 'dev')#load test set
    labels_to_ids = task5_labels_to_ids
    #input_data = (train_data, dev_data, labels_to_ids)

    #Define tokenizer, model and optimizer
    device = 'cuda' if cuda.is_available() else 'cpu' #save the processing time
    if model_load_flag:
        tokenizer = AutoTokenizer.from_pretrained(model_load_location)
        model = AutoModelForSequenceClassification.from_pretrained(model_load_location)
    else: 
        tokenizer =  AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(labels_to_ids))
    optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
    model.to(device)

    #Get dataloaders
    train_loader = initialize_data(tokenizer, initialization_input, train_data, labels_to_ids, shuffle = True)
    dev_loader = initialize_data(tokenizer, initialization_input, dev_data, labels_to_ids, shuffle = True)
    #test_loader = initialize_data(tokenizer, initialization_input, test_data, labels_to_ids, shuffle = True)#create test loader

    best_dev_acc = 0
    best_test_acc = 0
    best_dev_precision = 0
    best_test_precision = 0
    best_dev_recall = 0
    best_test_recall = 0
    best_dev_f1score = 0
    best_test_f1score = 0
    best_epoch = -1
    
    list_dev_acc = [] 
    list_test_acc = []  
    list_dev_precision = []  
    list_test_precision  = []  
    list_dev_recall = []  
    list_test_recall = []  
    list_dev_f1score = []  
    list_test_f1score = []
    
    for epoch in range(n_epochs):
        start = time.time()
        print(f"Training epoch: {epoch + 1}")

        #train model
        if not model_load_flag:
            model = train(epoch, train_loader, model, optimizer, device, grad_step)
        
        #testing and logging
        labels_dev, predictions_dev, dev_accuracy, dev_precision, dev_recall, dev_f1score = testing(model, dev_loader, labels_to_ids, device)
        print('DEV ACC:', dev_accuracy)
        print('DEV Precision:' , dev_precision)
        print('DEV Recall:' , dev_recall)
        print('DEV F1Score:' , dev_f1score)
        
        list_dev_acc.append(dev_accuracy)     
        list_dev_precision.append(dev_precision)   
        list_dev_recall.append(dev_recall)  
        list_dev_f1score.append(dev_f1score)  
        
        
        #labels_test, predictions_test, test_accuracy, test_precision, test_recall, test_f1score = testing(model, test_loader, labels_to_ids, device)
        #print('TEST ACC:', test_accuracy)
        #print('TEST Precision:' , test_precision)
        #print('TEST Recall:' , test_recall)
        #print('TEST F1Score:' , test_f1score)
        
        #list_test_acc.append(test_accuracy) 
        #list_test_precision.append(test_precision)  
        #list_test_recall.append(test_recall)
        #list_test_f1score.append(test_f1score) 

        #saving model
        if dev_accuracy > best_dev_acc:
            best_dev_acc = dev_accuracy
            #best_test_acc = test_accuracy
        if dev_precision > best_dev_precision:
            best_dev_precision = dev_precision
            #best_test_precision = test_precision
        if dev_recall > best_dev_recall:
            best_dev_recall = dev_recall
            #best_test_recall = test_recall
        if dev_f1score > best_dev_f1score:
            best_dev_f1score = dev_f1score
            #best_test_f1score = test_f1score
            best_epoch = epoch
            
            if model_save_flag:
                os.makedirs(model_save_location, exist_ok=True)
                tokenizer.save_pretrained(model_save_location)
                model.save_pretrained(model_save_location)

        now = time.time()
        print('BEST ACCURACY --> ', 'DEV:', round(best_dev_acc, 5))
        print('BEST PRECISION --> ', 'DEV:', round(best_dev_precision, 5))
        print('BEST RECALL --> ', 'DEV:', round(best_dev_recall, 5))
        print('BEST F1SCORE --> ', 'DEV:', round(best_dev_f1score, 5))
        print('TIME PER EPOCH:', (now-start)/60 )
        print()

    return best_dev_acc, best_test_acc, best_epoch, best_dev_precision, best_test_precision, best_dev_recall, best_test_recall, best_dev_f1score, best_test_f1score, list_dev_acc, list_test_acc, list_dev_precision, list_test_precision, list_dev_recall, list_test_recall, list_dev_f1score, list_test_f1score

In [5]:
if __name__ == '__main__':
    n_epochs = 10
    models = ['dccuchile/bert-base-spanish-wwm-cased']
    
    #model saving parameters
    model_save_flag = True
    model_load_flag = False
    
    overall_list_dev_acc = [] 
    overall_list_test_acc = []    
    overall_list_dev_precision = []  
    overall_list_test_precision  = []  
    overall_list_dev_recall = []  
    overall_list_test_recall = []  
    overall_list_dev_f1score = []  
    overall_list_test_f1score = [] 
    
    for i in range(5):
        
        for model_name in models:

            model_save_location = 'saved_models/' + model_name + '_over_under_sampled' + str(i)
            model_load_location = None

            best_dev_acc, best_test_acc, best_epoch, best_dev_precision, best_test_precision, best_dev_recall, best_test_recall, best_dev_f1score, best_test_f1score, list_dev_acc, list_test_acc, list_dev_precision, list_test_precision, list_dev_recall, list_test_recall, list_dev_f1score, list_test_f1score = main(n_epochs, model_name, model_save_flag, model_save_location, model_load_flag, model_load_location)
            
            overall_list_dev_acc.append(list_dev_acc) 
            overall_list_test_acc.append(list_test_acc) 
            overall_list_dev_precision.append(list_dev_precision)  
            overall_list_test_precision.append(list_test_precision) 
            overall_list_dev_recall.append(list_dev_recall)  
            overall_list_test_recall.append(list_test_recall)  
            overall_list_dev_f1score.append(list_dev_f1score)  
            overall_list_test_f1score.append(list_test_f1score) 

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchi

Training epoch: 1
FINSIHED BATCH: 19 of 330
FINSIHED BATCH: 39 of 330
FINSIHED BATCH: 59 of 330
FINSIHED BATCH: 79 of 330
FINSIHED BATCH: 99 of 330
FINSIHED BATCH: 119 of 330
FINSIHED BATCH: 139 of 330
FINSIHED BATCH: 159 of 330
FINSIHED BATCH: 179 of 330
FINSIHED BATCH: 199 of 330
FINSIHED BATCH: 219 of 330
FINSIHED BATCH: 239 of 330
FINSIHED BATCH: 259 of 330
FINSIHED BATCH: 279 of 330
FINSIHED BATCH: 299 of 330
FINSIHED BATCH: 319 of 330
Validation loss per 100 evaluation steps: 0.4321795701980591
DEV ACC: 0.8141166960611406
DEV Precision: 0.6437280972995257
DEV Recall: 0.8827223481985388
DEV F1Score: 0.7250477244838144
BEST ACCURACY -->  DEV: 0.81412
BEST PRECISION -->  DEV: 0.64373
BEST RECALL -->  DEV: 0.88272
BEST F1SCORE -->  DEV: 0.72505
TIME PER EPOCH: 7.911443245410919

Training epoch: 2
FINSIHED BATCH: 19 of 330
FINSIHED BATCH: 39 of 330
FINSIHED BATCH: 59 of 330
FINSIHED BATCH: 79 of 330
FINSIHED BATCH: 99 of 330
FINSIHED BATCH: 119 of 330
FINSIHED BATCH: 139 of 330
FINSIH

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchi

Training epoch: 1
FINSIHED BATCH: 19 of 330
FINSIHED BATCH: 39 of 330
FINSIHED BATCH: 59 of 330
FINSIHED BATCH: 79 of 330
FINSIHED BATCH: 99 of 330
FINSIHED BATCH: 119 of 330
FINSIHED BATCH: 139 of 330
FINSIHED BATCH: 159 of 330
FINSIHED BATCH: 179 of 330
FINSIHED BATCH: 199 of 330
FINSIHED BATCH: 219 of 330
FINSIHED BATCH: 239 of 330
FINSIHED BATCH: 259 of 330
FINSIHED BATCH: 279 of 330
FINSIHED BATCH: 299 of 330
FINSIHED BATCH: 319 of 330
Validation loss per 100 evaluation steps: 0.4088028073310852
DEV ACC: 0.8150169018224575
DEV Precision: 0.6097403478355862
DEV Recall: 0.9225875535399345
DEV F1Score: 0.7153281622358724
BEST ACCURACY -->  DEV: 0.81502
BEST PRECISION -->  DEV: 0.60974
BEST RECALL -->  DEV: 0.92259
BEST F1SCORE -->  DEV: 0.71533
TIME PER EPOCH: 8.089660863081614

Training epoch: 2
FINSIHED BATCH: 19 of 330
FINSIHED BATCH: 39 of 330
FINSIHED BATCH: 59 of 330
FINSIHED BATCH: 79 of 330
FINSIHED BATCH: 99 of 330
FINSIHED BATCH: 119 of 330
FINSIHED BATCH: 139 of 330
FINSIH

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchi

Training epoch: 1
FINSIHED BATCH: 19 of 330
FINSIHED BATCH: 39 of 330
FINSIHED BATCH: 59 of 330
FINSIHED BATCH: 79 of 330
FINSIHED BATCH: 99 of 330
FINSIHED BATCH: 119 of 330
FINSIHED BATCH: 139 of 330
FINSIHED BATCH: 159 of 330
FINSIHED BATCH: 179 of 330
FINSIHED BATCH: 199 of 330
FINSIHED BATCH: 219 of 330
FINSIHED BATCH: 239 of 330
FINSIHED BATCH: 259 of 330
FINSIHED BATCH: 279 of 330
FINSIHED BATCH: 299 of 330
FINSIHED BATCH: 319 of 330
Validation loss per 100 evaluation steps: 0.3288379907608032
DEV ACC: 0.8179930922986479
DEV Precision: 0.6278695342420832
DEV Recall: 0.9317964222726127
DEV F1Score: 0.7277512751841154
BEST ACCURACY -->  DEV: 0.81799
BEST PRECISION -->  DEV: 0.62787
BEST RECALL -->  DEV: 0.9318
BEST F1SCORE -->  DEV: 0.72775
TIME PER EPOCH: 8.061056268215179

Training epoch: 2
FINSIHED BATCH: 19 of 330
FINSIHED BATCH: 39 of 330
FINSIHED BATCH: 59 of 330
FINSIHED BATCH: 79 of 330
FINSIHED BATCH: 99 of 330
FINSIHED BATCH: 119 of 330
FINSIHED BATCH: 139 of 330
FINSIHE

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchi

Training epoch: 1
FINSIHED BATCH: 19 of 330
FINSIHED BATCH: 39 of 330
FINSIHED BATCH: 59 of 330
FINSIHED BATCH: 79 of 330
FINSIHED BATCH: 99 of 330
FINSIHED BATCH: 119 of 330
FINSIHED BATCH: 139 of 330
FINSIHED BATCH: 159 of 330
FINSIHED BATCH: 179 of 330
FINSIHED BATCH: 199 of 330
FINSIHED BATCH: 219 of 330
FINSIHED BATCH: 239 of 330
FINSIHED BATCH: 259 of 330
FINSIHED BATCH: 279 of 330
FINSIHED BATCH: 299 of 330
FINSIHED BATCH: 319 of 330
Validation loss per 100 evaluation steps: 0.47750335931777954
DEV ACC: 0.769510582010582
DEV Precision: 0.5855406057787013
DEV Recall: 0.8970206601158981
DEV F1Score: 0.6955678453172185
BEST ACCURACY -->  DEV: 0.76951
BEST PRECISION -->  DEV: 0.58554
BEST RECALL -->  DEV: 0.89702
BEST F1SCORE -->  DEV: 0.69557
TIME PER EPOCH: 8.044510861237844

Training epoch: 2
FINSIHED BATCH: 19 of 330
FINSIHED BATCH: 39 of 330
FINSIHED BATCH: 59 of 330
FINSIHED BATCH: 79 of 330
FINSIHED BATCH: 99 of 330
FINSIHED BATCH: 119 of 330
FINSIHED BATCH: 139 of 330
FINSIH

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchi

Training epoch: 1
FINSIHED BATCH: 19 of 330
FINSIHED BATCH: 39 of 330
FINSIHED BATCH: 59 of 330
FINSIHED BATCH: 79 of 330
FINSIHED BATCH: 99 of 330
FINSIHED BATCH: 119 of 330
FINSIHED BATCH: 139 of 330
FINSIHED BATCH: 159 of 330
FINSIHED BATCH: 179 of 330
FINSIHED BATCH: 199 of 330
FINSIHED BATCH: 219 of 330
FINSIHED BATCH: 239 of 330
FINSIHED BATCH: 259 of 330
FINSIHED BATCH: 279 of 330
FINSIHED BATCH: 299 of 330
FINSIHED BATCH: 319 of 330
Validation loss per 100 evaluation steps: 0.44753599166870117
DEV ACC: 0.8065843621399177
DEV Precision: 0.6918332989761561
DEV Recall: 0.7881834215167549
DEV F1Score: 0.7210814240226004
BEST ACCURACY -->  DEV: 0.80658
BEST PRECISION -->  DEV: 0.69183
BEST RECALL -->  DEV: 0.78818
BEST F1SCORE -->  DEV: 0.72108
TIME PER EPOCH: 7.69690500497818

Training epoch: 2
FINSIHED BATCH: 19 of 330
FINSIHED BATCH: 39 of 330
FINSIHED BATCH: 59 of 330
FINSIHED BATCH: 79 of 330
FINSIHED BATCH: 99 of 330
FINSIHED BATCH: 119 of 330
FINSIHED BATCH: 139 of 330
FINSIH

In [6]:
print(best_dev_acc)

0.8285016166960611


In [7]:
print(best_epoch)

0


In [8]:
print(best_dev_precision)

0.6918332989761561


In [9]:
print(best_dev_recall)

0.8557760141093474


In [10]:
print(best_dev_f1score)

0.7210814240226004


In [11]:
print(overall_list_dev_acc)

[[0.8141166960611406, 0.8192607289829512, 0.8059046149323927, 0.7978395061728395, 0.812444885361552, 0.7940917107583775, 0.8119488536155203, 0.8176807760141093, 0.8009442974720753, 0.7994562022339801], [0.8150169018224575, 0.8071722516166961, 0.8082929159318047, 0.8116365373309818, 0.8161008230452675, 0.8091563786008231, 0.8156047912992358, 0.8023037918871252, 0.819077013521458, 0.8125367430922987], [0.8179930922986479, 0.8143004115226337, 0.8168724279835391, 0.8041409465020576, 0.8080724573780129, 0.8051881246325692, 0.8025242504409171, 0.802120076425632, 0.8143371546149324, 0.8051881246325692], [0.769510582010582, 0.8233575837742505, 0.8090645208700764, 0.8000073486184597, 0.7933201058201058, 0.7997317754262199, 0.7971965020576132, 0.8065292475014698, 0.8051881246325692, 0.8086603468547914], [0.8065843621399177, 0.8285016166960611, 0.8045083774250441, 0.8126286008230453, 0.8183605232216343, 0.8091563786008231, 0.8178093768371546, 0.7844466490299823, 0.7866512345679012, 0.809468694885

In [12]:
print(overall_list_dev_precision)

[[0.6437280972995257, 0.6389787901692663, 0.6567838246409676, 0.6765529444100873, 0.6546233308138072, 0.652166792642983, 0.6342632675966007, 0.6569790879314689, 0.6568697175840031, 0.6659709338280767], [0.6097403478355862, 0.6551461325270851, 0.636827678494345, 0.6362153278819948, 0.6520997961474153, 0.6469891660367851, 0.6620433358528598, 0.6560669048764284, 0.6938082640463594, 0.6350640981593363], [0.6278695342420832, 0.6429049345716012, 0.617576559243226, 0.6616573902288189, 0.6213781809019905, 0.578140245997389, 0.6169730181634943, 0.6759179092512426, 0.6198538674729153, 0.6036373679230823], [0.5855406057787013, 0.6872849813326003, 0.6304076875505447, 0.6109032501889643, 0.6740236835474933, 0.6355820105820108, 0.6277903754094231, 0.6504208754208755, 0.6195996243615289, 0.6273906869144965], [0.6918332989761561, 0.6443191288429385, 0.6383935729173823, 0.6353111615016378, 0.6414003985432558, 0.626396046634142, 0.647439817677913, 0.6284202569916856, 0.6120708215946312, 0.63839758125472

In [13]:
print(overall_list_dev_recall)

[[0.8827223481985388, 0.8126354245401862, 0.6116717057193249, 0.6034832451499117, 0.8223544973544972, 0.5406399596875788, 0.7949168556311413, 0.7566326530612244, 0.6162784305641449, 0.6828666483428386], [0.9225875535399345, 0.7007810531620058, 0.7557067271352982, 0.7593262557548273, 0.7843348450491306, 0.6785691380929475, 0.7093829450972309, 0.7197908793146889, 0.6891070569642, 0.7485323759133283], [0.9317964222726127, 0.7529958137100995, 0.8122804003756386, 0.680971277399849, 0.6464823976728739, 0.723267825648778, 0.6872826908541193, 0.628776426395474, 0.8267573696145125, 0.7397077349458302], [0.8970206601158981, 0.7450617283950617, 0.732027187979569, 0.5572751322751323, 0.5499118165784832, 0.663432511051559, 0.651218578599531, 0.7400919627110102, 0.7640497950021762, 0.7047275475846904], [0.7881834215167549, 0.8557760141093474, 0.787459057697153, 0.737166449071211, 0.8149596875787348, 0.7125661375661373, 0.7979809432190385, 0.5393074738312833, 0.7084530108339633, 0.7368228772990677]]


In [14]:
print(overall_list_dev_f1score)

[[0.7250477244838144, 0.6981709188807816, 0.611411436260323, 0.6157595490928824, 0.7047696133756874, 0.5786304628041321, 0.6854547184268543, 0.6823379536522686, 0.6117631978976517, 0.6517841202542085], [0.7153281622358724, 0.6548427473608073, 0.6724396033580756, 0.6693912065023666, 0.683705210842908, 0.6469594295003404, 0.6601667309871646, 0.6617204487542094, 0.6715282098335317, 0.6645675061420305], [0.7277512751841154, 0.6791466992336814, 0.6816746889870874, 0.6550230814936698, 0.6163843054458338, 0.6281819607707858, 0.6248857174516911, 0.6087191364502288, 0.6927338910045674, 0.647925634382217], [0.6955678453172185, 0.6874764757523122, 0.6550309711896541, 0.5709181888379883, 0.5790054172407113, 0.621155146855427, 0.6180513080471308, 0.6679666898043719, 0.6727881294268488, 0.6411716301894111], [0.7210814240226004, 0.7189535637862334, 0.6784689190225087, 0.6598233033586122, 0.6971627309053353, 0.6500929951173207, 0.6967653677048646, 0.5640753892854734, 0.6384126641018871, 0.657515581103

In [15]:
#The best model is 2